In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from src.wrangle import read_and_filter_data
from datetime import datetime, timedelta

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 200)

In [2]:
data = pd.read_csv('city_payroll_data.csv', engine='pyarrow')

In [3]:
conditions = (
    (data['Agency Name'] == 'DEPT OF ED PEDAGOGICAL') &
    (data['Title Description'] == 'TEACHER') &
    (data['Leave Status as of June 30'] == 'ACTIVE') &
    (data['Regular Gross Paid'] > 0)
)

# Drop unused columns
df = data[conditions].drop(columns=['Payroll Number', 'Agency Name', 'Work Location Borough',
                                'Title Description', 'Pay Basis', 'Regular Hours', 'OT Hours',
                                'Total OT Paid', ]).drop_duplicates().reset_index(drop=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519308 entries, 0 to 519307
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Fiscal Year                 519308 non-null  int64  
 1   Last Name                   519308 non-null  object 
 2   First Name                  519308 non-null  object 
 3   Mid Init                    519308 non-null  object 
 4   Agency Start Date           519308 non-null  object 
 5   Leave Status as of June 30  519308 non-null  object 
 6   Base Salary                 519308 non-null  float64
 7   Regular Gross Paid          519308 non-null  float64
 8   Total Other Pay             519308 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 35.7+ MB


In [5]:
df.head()

,Fiscal Year,Last Name,First Name,Mid Init,Agency Start Date,Leave Status as of June 30,Base Salary,Regular Gross Paid,Total Other Pay
0,2022,APPELMAN,BROOKE,A,10/20/2015,ACTIVE,97469.0,90693.25,1486.01
1,2022,ARCIERO,DENISE,A,08/30/2007,ACTIVE,100351.0,100350.96,4851.73
2,2022,ARCILA,MARIA,I,02/02/2022,ACTIVE,69866.0,26274.50,2801.02
3,2022,ARCINIEGAS,KARINA,B,09/06/2005,ACTIVE,109852.0,104620.70,18903.82
4,2022,ARIAS,BEATRIZ,,09/05/2002,ACTIVE,111350.0,110164.13,14478.55


In [6]:
df[df['Last Name']=='APPELMAN'].sort_values(by='Fiscal Year')

,Fiscal Year,Last Name,First Name,Mid Init,Agency Start Date,Leave Status as of June 30,Base Salary,Regular Gross Paid,Total Other Pay
105980,2016,APPELMAN,BROOKE,A,10/20/2015,ACTIVE,59088.0,31363.88,6.99
263632,2017,APPELMAN,BROOKE,A,10/20/2015,ACTIVE,62233.0,56385.76,0.00
362272,2018,APPELMAN,BROOKE,A,10/20/2015,ACTIVE,66200.0,62787.13,0.00
414099,2019,APPELMAN,BROOKE,A,10/20/2015,ACTIVE,68270.0,68338.10,0.00
43762,2020,APPELMAN,BROOKE,A,10/20/2015,ACTIVE,71963.0,68945.79,1645.41
95231,2021,APPELMAN,BROOKE,A,10/20/2015,ACTIVE,81358.0,74186.53,712.93
0,2022,APPELMAN,BROOKE,A,10/20/2015,ACTIVE,97469.0,90693.25,1486.01
471945,2023,APPELMAN,BROOKE,A,10/20/2015,ACTIVE,97469.0,97469.04,475.00


In [7]:
df.rename(columns={'Agency Start Date': 'Hire Date',
                'Base Salary': 'Salary'}, inplace=True)

# Cast Hire Date to datetime and add Hire Year
df['Hire Date'] = pd.to_datetime(df['Hire Date'], errors='coerce')
df = df.dropna(subset=['Hire Date'])
df['Hire Year'] = df['Hire Date'].dt.year
df['Hire Year'] = df['Hire Year'].astype('Int16')

# Calculate the number of years employed as a NYC teacher
df['Years of Employment'] = df['Fiscal Year'] - df['Hire Year']
df['Years of Employment'] = df['Years of Employment'].astype('Int16')
df['Years of Employment'] = pd.to_numeric(df['Years of Employment'], errors='coerce')
df = df.dropna(subset=['Years of Employment'])

# Employee Key
df[['Last Name', 'First Name', 'Mid Init']] = df[['Last Name', 'First Name', 'Mid Init']].apply(
    lambda x: x.str.strip().str.title().fillna('None')
)
df['FirstMidLastStart'] = df['First Name'] + df['Mid Init'] + df['Last Name'] + df['Hire Date'].astype(str)
df['Employee ID'], _ = pd.factorize(df['FirstMidLastStart'], sort=True)
df = df.drop(columns=['FirstMidLastStart', 'Last Name', 'First Name', 'Mid Init'])

df = df.sort_values(by=['Employee ID', 'Fiscal Year']).reset_index(drop=True)

# Salary changes YoY
df['Salary Delta'] = df.groupby(by=['Employee ID'])['Salary'].pct_change() * 100
df['Salary Monetary Diff'] = df.groupby(by=['Employee ID'])['Salary'].diff()
df['Other Pay Delta'] = (df.groupby(by=['Employee ID'])['Total Other Pay'].pct_change() * 100).fillna(0).round(2)
df['Other Pay Monetary Diff'] = (df.groupby(by=['Employee ID'])['Total Other Pay'].diff()).fillna(0).round(2)

df[['Salary Delta','Salary Monetary Diff']] = (df[['Salary Delta','Salary Monetary Diff']]
                                                .fillna(0)
                                                .round(2)
                                                )

# Remove outliers
df = df[(df['Hire Year']>=1980)&
        (df['Years of Employment']<=50)&
        (df['Fiscal Year']>2014)
        ].reset_index(drop=True)

In [8]:
df.head()

,Fiscal Year,Hire Date,Leave Status as of June 30,Salary,Regular Gross Paid,Total Other Pay,Hire Year,Years of Employment,Employee ID,Salary Delta,Salary Monetary Diff,Other Pay Delta,Other Pay Monetary Diff
0,2018,2011-09-01,ACTIVE,71930.0,65187.45,0.00,2011,7,0,0.00,0.0,0.00,0.00
1,2019,2011-09-01,ACTIVE,79005.0,80565.40,0.00,2011,8,0,9.84,7075.0,0.00,0.00
2,2020,2011-09-01,ACTIVE,89263.0,83817.77,5049.65,2011,9,0,12.98,10258.0,inf,5049.65
3,2021,2011-09-01,ACTIVE,91941.0,89597.73,15754.28,2011,10,0,3.00,2678.0,211.99,10704.63
4,2022,2011-09-01,ACTIVE,91941.0,91941.12,26024.37,2011,11,0,0.00,0.0,65.19,10270.09


In [9]:
df = df.drop(columns=['Leave Status as of June 30'])

In [10]:
df = df[['Fiscal Year',
 'Employee ID',
 'Hire Date',
 'Hire Year',
 'Years of Employment',
 'Salary',
 'Total Other Pay',
 'Salary Delta',
 'Other Pay Delta',
 'Salary Monetary Diff',
 'Other Pay Monetary Diff'
 ]]

In [11]:
df[df['Employee ID']==18992]

,Fiscal Year,Employee ID,Hire Date,Hire Year,Years of Employment,Salary,Total Other Pay,Salary Delta,Other Pay Delta,Salary Monetary Diff,Other Pay Monetary Diff
94467,2018,18992,2017-11-01,2017,1,56711.0,0.00,0.00,0.00,0.0,0.00
94468,2019,18992,2017-11-01,2017,2,58995.0,0.00,4.03,0.00,2284.0,0.00
94469,2020,18992,2017-11-01,2017,3,60970.0,14158.64,3.35,inf,1975.0,14158.64
94470,2021,18992,2017-11-01,2017,4,63708.0,14168.21,4.49,0.07,2738.0,9.57
94471,2022,18992,2017-11-01,2017,5,81022.0,15293.55,27.18,7.94,17314.0,1125.34
94472,2023,18992,2017-11-01,2017,6,91941.0,94466.93,13.48,517.69,10919.0,79173.38


In [12]:
df_uft_negotiated_contract = df[df['Fiscal Year']>=2021]

In [13]:
df_schedule = pd.read_csv('salary_schedule_2021.csv', engine='pyarrow')

In [14]:
df_schedule

,Paystep,BAC1,C1+PD,BA+30C2,C2+ID,C2+PD,C2+ID+PD,MA+30C6,MA+30C6+PD
0,1A,61070,66601,63120,66909,68652,72437,70703,76231
1,1B,61070,66601,63120,66909,68652,72437,70703,76231
2,2A,62284,67815,64334,68123,69866,73651,71917,77445
3,2B,62284,67815,64334,68123,69866,73651,71917,77445
4,3A,62799,68330,64849,68638,70381,74166,72432,77960
5,3B,62799,68330,64849,68638,70381,74166,72432,77960
6,4A,63708,69239,65758,69547,71290,75075,73341,78869
7,4B,63708,69239,65758,69547,71290,75075,73341,78869
8,5A,64494,70025,66544,70333,72076,75861,74127,79655
9,5B,64494,70025,66544,70333,72076,75861,74127,79655


In [15]:
def get_paystep(salary):
    for index, row in df_schedule.iterrows():
        for col in df_schedule.columns[1:]:
            if salary == row[col]:
                return row['Paystep']
    return None

In [16]:
df_uft_negotiated_contract['Paystep'] = df_uft_negotiated_contract['Salary'].astype('int').apply(get_paystep)

C:\Users\chris\AppData\Local\Temp\ipykernel_21668\2052967809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uft_negotiated_contract['Paystep'] = df_uft_negotiated_contract['Salary'].astype('int').apply(get_paystep)


In [17]:
df_uft_negotiated_contract

,Fiscal Year,Employee ID,Hire Date,Hire Year,Years of Employment,Salary,Total Other Pay,Salary Delta,Other Pay Delta,Salary Monetary Diff,Other Pay Monetary Diff,Paystep
3,2021,0,2011-09-01,2011,10,91941.0,15754.28,3.00,211.99,2678.0,10704.63,8B+L5
4,2022,0,2011-09-01,2011,11,91941.0,26024.37,0.00,65.19,0.0,10270.09,8B+L5
10,2021,2,1996-09-03,1996,25,128657.0,5995.18,3.00,-50.92,3748.0,-6220.47,8B+L22
11,2022,2,1996-09-03,1996,26,128657.0,6449.36,0.00,7.58,0.0,454.18,8B+L22
12,2023,2,1996-09-03,1996,27,128657.0,475.00,0.00,-92.63,0.0,-5974.36,8B+L22
...,...,...,...,...,...,...,...,...,...,...,...,...
465853,2021,91953,2008-08-28,2008,13,101441.0,5052.57,3.00,-50.70,2955.0,-5196.38,8B+L10
465854,2022,91953,2008-08-28,2008,14,104145.0,5556.54,2.67,9.97,2704.0,503.97,8B+L13
465855,2023,91953,2008-08-28,2008,15,104145.0,173.23,0.00,-96.88,0.0,-5383.31,8B+L13
465862,2022,91955,2008-08-28,2008,14,104145.0,14494.02,5.75,64.23,5659.0,5668.63,8B+L13


In [18]:
def get_differential(salary):
    for index, row in df_schedule.iterrows():
        for col in df_schedule.columns[1:]:
            if salary == row[col]:
                return col
    return None

In [19]:
df_uft_negotiated_contract['Differential'] = df_uft_negotiated_contract['Salary'].astype('int').apply(get_differential)

In [ ]:
df_uft_negotiated_contract['Paystep'].value_counts()